In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval

pd.set_option('display.max_columns', 70)                       # 출력할 열의 최대개수
pd.set_option('display.max_colwidth', 80)                      # 출력할 열의 너비

In [2]:
# 필요한 데이터 불러오기
data1 = pd.read_csv('data_for_service_1.csv')
data2 = pd.read_excel('place-data-v.1.3.xlsx')
sim_order_table = pd.read_csv('유사도 순으로 장소 인덱스를 나열_10개만.csv')

In [4]:
# 여행 키워드 단어와 가중치를 입력하시오!
# 단 키워드 단어는 
# ['돈','쇼핑','시장','싸다, 저렴하다','비싸다','먹다','음식, 식사','상품','구매',
# '가격','호텔','박물관','전시관','관광지','유적지','미술관','역사','유익하다','문화','궁전','전쟁','활동','놀이공원','게임',
# '친구','커플','재미','아쿠아리움','엔터테인먼트','아이','가족','경기','힐링','공원','휴식','풍경','자연','전망대','산책',
# '분수','여유','나들이','공연','콘서트','이벤트','영화','예술','미술','강좌','요리','공예','체험'] 중에서 선택해야한다.

# 대략 5개의 키워드가 입력되었을 경우(실제로는 이렇게 안 되겠지만, 5개 상위 키워드로 뽑아서 진행할 것임)

keyword1 = input('첫 번째 여행 키워드는? : ')
weight1 = input('위 키워드의 가중치는? : ')
keyword2 = input('두 번째 여행 키워드는? : ')
weight2 = input('위 키워드의 가중치는? : ')
keyword3 = input('세 번째 여행 키워드는? : ')
weight3 = input('위 키워드의 가중치는? : ')
keyword4 = input('네 번째 여행 키워드는? : ')
weight4 = input('위 키워드의 가중치는? : ')
keyword5 = input('5 번째 여행 키워드는? : ')
weight5 = input('위 키워드의 가중치는? : ')

keyword = [keyword1, keyword2, keyword3, keyword4, keyword5]
weight = [int(weight1), int(weight2), int(weight3), int(weight4), int(weight5)]

user_data = pd.DataFrame({'keyword':keyword, 'weight':weight})
user_data['rate'] = user_data['weight']/(sum(user_data['weight']))
user_data = user_data.sort_values('rate', ascending = False)
user_data = user_data.reset_index()
print(user_data)

첫 번째 여행 키워드는? : 전시관
위 키워드의 가중치는? : 15
두 번째 여행 키워드는? : 여유
위 키워드의 가중치는? : 20
세 번째 여행 키워드는? : 먹다
위 키워드의 가중치는? : 15
네 번째 여행 키워드는? : 힐링
위 키워드의 가중치는? : 4
5 번째 여행 키워드는? : 아이
위 키워드의 가중치는? : 10
   index keyword  weight      rate
0      1      여유      20  0.312500
1      0     전시관      15  0.234375
2      2      먹다      15  0.234375
3      4      아이      10  0.156250
4      3      힐링       4  0.062500


In [25]:
# 여행자 상의 두 타입 고르기
# def tour_type_1and2_with_weight(user_data):

tour_type_Table = {"tour_type" : ["Flex", "교양인", "놀래", "쉴래", "구경할래", "배울래"],
                   "sum_weight": [0,0,0,0,0,0]}
tour_type_Table = pd.DataFrame(tour_type_Table)

flex = 0
smart = 0
hangout = 0
rest = 0
sightseeing = 0
learning = 0

for i in range(len(user_data)):
    key_text = user_data.iloc[i,1]
    key_weight = user_data.iloc[i,3]

    if key_text in '돈, 쇼핑, 시장, 싸다, 저렴하다, 비싸다,먹다,음식, 식사, 상품,구매':
        flex +=key_weight
    if key_text in '박물관, 전시관,관광지,유적지,역사,유익하다,문화,궁전,전쟁':
        smart +=key_weight
    if key_text == '미술관':
        smart +=key_weight
    if key_text in '활동,놀이공원,게임,친구,커플,재미,아쿠아리움,엔터테인먼트,아이,가족,경기':
        hangout +=key_weight 
    if key_text in '힐링,공원,휴식,풍경,자연,전망대,산책,분수,여유,나들이':
        rest +=key_weight 
    if key_text in '공연,콘서트,이벤트,영화,예술,미술':
        sightseeing +=key_weight 
    if key_text in '강좌,요리,공예,체험':
        learning +=key_weight

tour_type_Table['sum_weight'] = [flex, smart, hangout, rest, sightseeing, learning]
tour_type_Table = tour_type_Table.sort_values('sum_weight', ascending = False)
tour_type_Table = tour_type_Table.reset_index()

first_user_tour_type = tour_type_Table.iloc[0,1]
first_user_tour_type_weight = float(tour_type_Table.iloc[0,2]) * 100
first_user_tour_type_num = tour_type_Table.iloc[0,0] + 1
second_user_tour_type = tour_type_Table.iloc[1,1]
second_user_tour_type_weight = float(tour_type_Table.iloc[1,2]) * 100
second_user_tour_type_num = tour_type_Table.iloc[1,0] + 1
thrid_user_tour_type_num = tour_type_Table.iloc[2,0] + 1
fourth_user_tour_type_num = tour_type_Table.iloc[3,0] + 1

user_info = [first_user_tour_type,first_user_tour_type_weight,first_user_tour_type_num,
            second_user_tour_type,second_user_tour_type_weight,second_user_tour_type_num,
             thrid_user_tour_type_num, fourth_user_tour_type_num]

tour_type_Table

,index,tour_type,sum_weight
0,3,쉴래,0.375000
1,0,Flex,0.234375
2,1,교양인,0.234375
3,2,놀래,0.156250
4,4,구경할래,0.000000
5,5,배울래,0.000000


In [26]:
# 가상 장소 확정 짓기 (키워드 별로 뽑기)
keyword_order = list(user_data['keyword'])
print(keyword_order)

# 키워드가 들어간 장소는 다 뽑기(쉬운 모델링을 위해 상위 3개만 고려함....)
firstword_table = data1[data1['tag_list'].str.contains(keyword_order[0])]
secondword_table = data1[data1['tag_list'].str.contains(keyword_order[1])]
thirdword_table = data1[data1['tag_list'].str.contains(keyword_order[2])]

entire_table = pd.concat([firstword_table,secondword_table,thirdword_table], axis=0)
entire_table = entire_table.drop_duplicates()
print('모으기')
print('entire_table 개수는 : ', len(entire_table))

if len(entire_table) >= 100:
    next_table = firstword_table
else : next_table = entire_table

print('next_table 개수는 : ', len(next_table))

# 한 번 걸러진,next_table에서 우선순위 단어 고려해보기
A1 = next_table[next_table['first_tag'] == keyword_order[0]]
A2 = next_table[next_table['second_tag'] == keyword_order[0]]
A3 = next_table[next_table['third_tag'] == keyword_order[0]]
important_A = pd.concat([A1,A2,A3])
important_A = important_A.drop_duplicates()

if len(next_table) >= 50:
    semi_table = A1
elif len(next_table) >= 20:
    semi_table = important_A
else : semi_table = next_table

print('semi_table 개수는 : ', len(semi_table))

B1 = semi_table[semi_table['first_tag'] == keyword_order[1]]
B2 = semi_table[semi_table['second_tag'] == keyword_order[1]]
B3 = semi_table[semi_table['third_tag'] == keyword_order[1]]
B4 = semi_table[semi_table['fourth_tag'] == keyword_order[1]]
B5 = semi_table[semi_table['fifth_tag'] == keyword_order[1]]

important_B = pd.concat([B1,B2,B3,B4,B5])
important_B = important_B.drop_duplicates()

if len(important_B)<= 5:
    final_table =semi_table
elif len(semi_table) >=30:
    if len(important_B[(important_B['first_tag'] == keyword_order[0]) & (important_B['second_tag'] == keyword_order[1])]) >= 1:
        final_table = important_B[(important_B['first_tag'] == keyword_order[0]) & (important_B['second_tag'] == keyword_order[1])]
    else : final_table = important_B[(important_B['first_tag'] == keyword_order[0]) | (important_B['second_tag'] == keyword_order[1])]
elif len(semi_table) >=20:
    final_table = important_B[important_B['first_tag'] == keyword_order[0]]
else : final_table = semi_table

print('final_table 개수는 : ', len(final_table))
    
# 최종 spot 선정
# '투어' 들어간 데이터 제외시키기
tour_table = final_table[final_table['place'].str.contains('투어')]
tour_table_index = tour_table.index.to_list()
final_table = final_table.drop(tour_table_index, axis=0)
print(final_table)

if len(final_table) >=2:
    end_table = final_table.sort_values('count', ascending=False).iloc[:3,:]
else : end_table = final_table

# 최종 spot 장소( 3개 이하)
print(end_table)
print(len(end_table))

['여유', '전시관', '먹다', '아이', '힐링']
모으기
entire_table 개수는 :  465
next_table 개수는 :  30
semi_table 개수는 :  7
final_table 개수는 :  7
        id               place  \
98     100               서울책보고   
146    159           교복몰 - 사당점   
327    686              김종영미술관   
506    959  CGV Gonghang Store   
1038  2070            광나루 한강공원   
110    113    Gana Art Gallery   
414    854             메가박스 목동   

                                                tag_list  count  avg_score  \
98    ['여유', '문화', '싸다, 저렴하다', '비싸다', '아이', '관광지', '구매']      4   4.250000   
146                                         ['커플', '여유']      1   5.000000   
327                           ['전시관', '여유', '미술관', '미술']      1   5.000000   
506                                         ['영화', '여유']      1   3.000000   
1038                                  ['공원', '여유', '먹다']      2   4.500000   
110          ['예술', '전시관', '여유', '아이', '가족', '휴식', '자연']      3   4.666667   
414      ['영화', '쇼핑', '여유', '음식, 식사', '먹다', '미술', '미술관']   

In [27]:
# end_table이 완성이 되었다면, 여기서 1~3 개 정도의 spot을 가상으로 잡아, 유사도 높은 장소를 각가 2개씩 뽑아오기
# 원래는 이 과정에서 여행 타입 별로 (비율적으로 분할해야한다...)
virtual_place_name = []
virtual_place_id = []
virtual_place_index = end_table.index.to_list()

for i in range(len(end_table)):
    vname = end_table.iloc[i,1]
    vid = end_table.iloc[i,0]
    virtual_place_name.append(vname)
    virtual_place_id.append(vid)
print(virtual_place_name)
print(virtual_place_id)
print(virtual_place_index)

['메가박스 목동', '서울책보고', 'Gana Art Gallery']
[854, 100, 113]
[414, 98, 110]


In [28]:
# 각 장소 별 2군데 추가로 유사도로 선정

recommend_id = []
recommend_name = []
recommend_index = []
recommend_tourtype = []

for i in range(3):
    recom = sim_order_table[sim_order_table['0']== virtual_place_name[i]]
    spot1 = recom.iloc[0,1]
    spot2 = recom.iloc[0,2]
    spot3 = recom.iloc[0,3]
    spot4 = recom.iloc[0,4]
    spot5 = recom.iloc[0,5]
    spot6 = recom.iloc[0,6]
    spot7 = recom.iloc[0,7]
    spot8 = recom.iloc[0,8]
    spot9 = recom.iloc[0,9]
    
    recommend_index.append(virtual_place_index[i])
    recommend_index.append(spot1)
    recommend_index.append(spot2)
    recommend_index.append(spot3)
    recommend_index.append(spot4)
    recommend_index.append(spot5)
    recommend_index.append(spot6)
    recommend_index.append(spot7)
    recommend_index.append(spot8)
    recommend_index.append(spot9)

    
for j in recommend_index:
    find_name = sim_order_table.iloc[j,0]
    recommend_name.append(find_name)

for k in recommend_index:
    find_id = data1.iloc[k,0]
    recommend_id.append(find_id)

for t in recommend_id:
    find_type = list(data2[data2['id']== t]['tour_type'])[0]
    recommend_tourtype.append(find_type)
    
print(recommend_index)
print(recommend_name)
print(recommend_id)
print(recommend_tourtype)

[414, 90, 503, 422, 329, 437, 401, 417, 441, 952, 98, 154, 565, 578, 486, 89, 972, 213, 38, 214, 110, 859, 1018, 171, 327, 1080, 875, 1045, 1073, 230]
['메가박스 목동', '명가', '롯데시네마 홍대입구', '롯데시네마 김포공항', '토탈 미술관', '롯데시네마 가산디지털', 'CGV 영등포', 'CGV 목동', 'CGV 구로', '로드', '서울책보고', '장지방', 'Samsung Leports Center', '삼모스포렉스', '스타24시 동네볼링장', 'Eunnamu', 'After Dark', '청하 플러스', '동대문 밀리오레', '제일 자수정', 'Gana Art Gallery', '불암산', '꿈의 숲', 'Art Center Nabi', '김종영미술관', '파이어맨', 'hanbokdan', '사직 공원', '점프! 코믹 무술 퍼포먼스', '박물관은살아있다 인사동점']
[854, 92, 956, 862, 688, 878, 841, 857, 882, 1951, 100, 169, 1171, 1196, 934, 91, 1974, 256, 39, 257, 113, 1731, 2049, 193, 686, 2203, 1749, 2077, 2196, 586]
[5, 1, 5, 3, 2, 5, 5, 5, 5, 1, 4, 1, 1, 1, 3, 1, 1, 1, 1, 1, 5, 6, 4, 2, 2, 5, 6, 4, 5, 3]


In [21]:
# 최종 추천지 반환하기!!! (중복 당연히 제거)

final_recommend_id = []
final_recommend_name = []
final_recommend_tourtype = []
jungbok = []
jungsang = []

# 리뷰수가 5개 이상은 되는 곳으로!!
for a in recommend_id:
    if data1[data1['id'] == a].iloc[0,3]>=5:
        final_recommend_id.append(a)

# 중복제거를 위의 순서를 고려해서 하기!!(무작위 set안됨.)
for b in final_recommend_id:
    if b in jungsang:
        jungbok.append(b)
    else : jungsang.append(b)

for k in jungsang:
    final_name = data1[data1['id']==k].iloc[0,1]
    final_recommend_name.append(final_name)
    
for k in jungsang:
    final_tourtype = list(data2[data2['id']==k]['tour_type'])[0]
    final_recommend_tourtype.append(final_tourtype)

    
print("총 추천 결과는??") 
Recommendation_total = pd.DataFrame({'id':jungsang, 'name' : final_recommend_name, 'tour_type': final_recommend_tourtype})
Recommendation_total

총 추천 결과는??


,id,name,tour_type
0,854,메가박스 목동,5
1,862,롯데시네마 김포공항,3
2,878,롯데시네마 가산디지털,5
3,841,CGV 영등포,5
4,857,CGV 목동,5
5,882,CGV 구로,5
6,1951,로드,1
7,39,동대문 밀리오레,1
8,257,제일 자수정,1
9,2049,꿈의 숲,4


In [29]:
# 상위 두 타입에 맞게 소팅하기

# 상위 2개가 아닌 여행자 타입 구하기
for_extra = list(set(Recommendation_total['tour_type'].to_list()))
main = [user_info[2], user_info[5]]
for_extra = [x for x in for_extra if x not in main]

table1 = Recommendation_total[Recommendation_total['tour_type']==user_info[2]]
table2 = Recommendation_total[Recommendation_total['tour_type']==user_info[5]]
table3_extra = Recommendation_total[Recommendation_total['tour_type']==for_extra[0]]
table4_extra = Recommendation_total[Recommendation_total['tour_type']==for_extra[1]]

if len(table1)<=4:
    plus_index = 4-len(table1)
    plus = table3_extra.iloc[:int(plus_index)+1,:]
    final_table1 = pd.concat([table1,plus])
    if len(final_table1)<=4:
        plus_index_e = 4-len(final_table1)
        plus_e = table4_extra.iloc[:int(plus_index_e)+1,:]
        final_table1 = pd.concat([final_table1,plus_e])     
else : final_table1 = table1.iloc[:5,:]
    
if len(table2)<=4:
    plus_index = 4-len(table2)
    plus = table3_extra.iloc[:int(plus_index)+1,:]
    final_table2 = pd.concat([table2,plus])
    if len(final_table2)<=4:
        plus_index_e = 4-len(final_table2)
        plus_e = table4_extra.iloc[:int(plus_index_e)+1,:]
        final_table2 = pd.concat([final_table2,plus_e])     
else : final_table2 = table2.iloc[:5,:]

In [32]:
final_table1

,id,name,tour_type
9,2049,꿈의 숲,4
11,2077,사직 공원,4
1,862,롯데시네마 김포공항,3
13,586,박물관은살아있다 인사동점,3
0,854,메가박스 목동,5


In [33]:
final_table2

,id,name,tour_type
6,1951,로드,1
7,39,동대문 밀리오레,1
8,257,제일 자수정,1
1,862,롯데시네마 김포공항,3
13,586,박물관은살아있다 인사동점,3
